# Exploratory Data Analysis on No-Fly Zone Data
In this notebook, the dataset obtained from rijksoverheid.nl is analysed and prepared for analytical use in this study. Omdat "Alle statische zonering en andere gebieden in ED269 format" nog data mist van Natura 2000 areas (alleen het wadden gebied zit erin) hebben we een losse data set van de natura 2000 gemerged met data. Additionally, the Natura 2000 dataset is also analysed and merged with the government dataset.

In [ ]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import json
import osmnx as ox
from shapely.geometry import Point, Polygon
import folium
import shapely

In [ ]:
# Load the data
with open('../3.no_fly_zones/data/Alle statische zonering en andere gebieden in ED269 format - 22juli2024.JSON', "r", encoding="utf-8") as file:
    data = json.load(file)

natura2000_zones = gpd.read_file("../3.no_fly_zones/data/natura2000.geojson")
natura2000_zones = natura2000_zones.to_crs(epsg=4326)

## 1. Read data from Rijksoverheid

In [ ]:
# Read the GeoJSON structure
features = data.get("features", [])
if not isinstance(features, list):
    raise ValueError("The JSON structure does not contain a list of features.")

# Extract relevant information
geo_zones = []
for feature in features:
    identifier = feature.get("identifier")
    country = feature.get("country")
    name = feature.get("name")
    zone_type = feature.get("type")
    restriction = feature.get("restriction")
    reason = ", ".join(feature.get("reason", [])) if isinstance(feature.get("reason", []), list) else ""

    # Extract 'zoneAuthority' (list)
    zone_authority = feature.get("zoneAuthority", [{}])
    zone_authority_name = zone_authority[0].get("name", "") if isinstance(zone_authority, list) else ""
    zone_authority_email = zone_authority[0].get("email", "") if isinstance(zone_authority, list) else ""

    # Extract 'applicability'
    applicability = feature.get("applicability", [{}])
    permanent = applicability[0].get("permanent", "") if isinstance(applicability, list) else ""

    # Extract 'message'
    message = feature.get("message", "")

    # Extract 'geometry' (list)
    geometry_list = feature.get("geometry", [])
    
    # Process multiple geometries
    for geometry in geometry_list:
        upper_limit = geometry.get("upperLimit", None)
        lower_limit = geometry.get("lowerLimit", None)
        uom_dimensions = geometry.get("uomDimensions", None)
        upper_vertical_reference = geometry.get("upperVerticalReference", None)
        lower_vertical_reference = geometry.get("lowerVerticalReference", None)

        # Extract geometry type and coordinates
        horizontal_projection = geometry.get("horizontalProjection", {})
        geometry_type = horizontal_projection.get("type", "")
        coordinates = horizontal_projection.get("coordinates", [])
        center = horizontal_projection.get("center", None)
        radius = horizontal_projection.get("radius", None)

        # Default geometry
        shapely_geometry = None

        # Convert different geometry types to Shapely
        if geometry_type == "Polygon":
            if isinstance(coordinates, list) and len(coordinates) > 0:
                shapely_geometry = Polygon(coordinates[0])  # Eerste ring als Polygon
        elif geometry_type == "Circle":
            if isinstance(center, list) and len(center) == 2:
                shapely_geometry = Point(center)  # Bewaar het middelpunt als een punt

        geo_zones.append({
            "identifier": identifier,
            "country": country,
            "name": name,
            "type": zone_type,
            "restriction": restriction,
            "reason": reason,
            "zoneAuthority_name": zone_authority_name,
            "permanent": permanent,
            "message": message,
            "upper_limit": upper_limit,
            "lower_limit": lower_limit,
            "geometry_type": geometry_type,
            "geometry": shapely_geometry,  # Dit is nu of een Polygon of een Point
            "circle_radius": radius if geometry_type == "Circle" else None  # Houd de radius apart
        })

# Convert to DataFrame
df = pd.DataFrame(geo_zones)

In [ ]:
df

## 2. Read Natura2000 data and merge

In [ ]:
# Clean up the DataFrame
natura2000_zones.rename(columns={"id": "identifier", "naamN2K": "name", "status": "message"}, inplace=True)
natura2000_zones.drop(columns=["vhnNew", "nr", "beschermin", "sitecodeV", "sitecodeH", "kadaster", "staatscour"], inplace=True)

In [ ]:
# Add to natura2000_zones["name"] Natura 2000 Area
natura2000_zones["name"] = "Natura 2000 Area " + natura2000_zones["name"]

In [ ]:
# Add upper and lower limits to the DataFrame
natura2000_zones['upper_limit'] = 120
natura2000_zones['lower_limit'] = 0

In [ ]:
# Combine the two dataframes
df = pd.concat([df, natura2000_zones], ignore_index=True)

In [ ]:
df

## 3. EDA on resulting dataframe

In [ ]:
def EDA(column):
    print("Data type of", column, "column: ", df[column].dtype)
    print("NaN's in", column, "column: ", df[column].isna().sum())
    print("Unique values in", column, "column: ", len(df[column].unique()), "\n")
    print("Value counts of", column, "column: ")
    print(df[column].value_counts(), "\n")

In [ ]:
EDA('identifier')

In [ ]:
EDA('country')

In [ ]:
EDA('name')

In [ ]:
# Function to categorize air types based on the name
def air_type(name):
    name = name.lower()  # Convert to lowercase for consistent comparison

    # Trauma- en reddingshelikopter landingsplaatsen
    if "traumahelikopter" in name or "reddingshelikopter" in name or "TRAUMAHELICOPTER" in name:
        return "Air Ambulance Landing Sites"

    # Nabijheid van vliegvelden (CTR - Controlled Traffic Region)
    elif "ctr" in name:
        return "CTR Zones"

    elif "nabijheid van vliegveld" in name:
        return "Airports"

    # Defensiegebieden en militaire oefengebieden
    elif "defensiegebied" in name or "ehd" in name:
        return "General Training Area (EHD)"

    # Permanent gereserveerde luchtruimen (Restricted Airspace - EHR)
    elif "ehr" in name:
        return "Permanently Reserved Airspace (EHR)"

    # Tijdeijk gereserveerde luchtruimen (Temporary Reserved Airspace - EHTRA)
    elif "ehtra" in name:
        return "Temporarily Reserved Airspace (EHTRA)"

    # Test- en dronegebieden (Special Activity Airspace - EHTSA)
    elif "ehtsa" in name:
        return "Temporarily Segregated Airspace (EHTSA)"

    # Laagvliegzones (Military Low Flying Area - EHD)
    elif "laagvliegroute" in name or "laagvlieggebied" in name or name.startswith("glv"):
        return "Low-Flying Area (GLV) and low flight routes"

    # Havengebieden met risico’s op zware ongevallen
    elif "havengebied" in name or "industriegebied" in name:
        return "High-Risk Areas"

    # Gebieden met speciale beveiliging
    elif "beveiligingsoverwegingen" in name or "vitaal proces" in name or "vitale processen" in name:
        return "Restricted Zones"

    # Milieubeschermingsgebieden
    elif "natura" in name or "millieubeschermingsgebied" in name or "waddenzee" in name:
        return "Protected Nature Reserves"

    # Verboden gebieden voor luchtverkeer (EHP - Prohibited Airspace)
    elif "verboden gebied" in name or "ehp" in name:
        return "Prohibited Airspace (EHP)"

    # Oefengebieden voor externe blusinstallaties van helikopters
    elif "blusinstallatie" in name:
        return "Firefighting Training Areas"

    # Anders: onbekende categorie
    else:
        return "Unknown"

In [ ]:
df["air_type"] = df["name"].apply(air_type)

In [ ]:
df["air_type"].value_counts()

As discussed in the report, not all no-fly zones are treated as strictly prohibited areas in this study. Therefore, we distinguish between different types of no-fly zones.

In [ ]:
def no_fly_zones(air_type):
    if air_type == "Restricted Zones":
        return True
    
    elif air_type == "High-Risk Areas":
        return True
    
    elif air_type == "Protected Nature Reserves":
        return True

    elif air_type == "CTR Zones":
        return False

    elif air_type == "Air Ambulance Landing Sites":
        return True
    
    elif air_type == "Temporarily Segregated Airspace (EHTSA)":
        return False

    elif air_type == "Permanently Reserved Airspace (EHR)":
        return True

    elif air_type == "General Training Area (EHD)":
        return True

    elif air_type == "Low-Flying Area (GLV) and low flight routes":
        return False

    elif air_type == "Airports":
        return False

    elif air_type == "Temporarily Reserved Airspace (EHTRA)":
        return False

    elif air_type == "Prohibited Airspace (EHP)":
        return True

    elif air_type == "Firefighting Training Areas":
        return True

    else:
        return False
    

In [ ]:
df["no_fly"] = df["air_type"].apply(no_fly_zones)

In [ ]:
df['type'] = 'UAV geozone'

In [ ]:
df

### Now that both dataframes have been merged, we will explore the structure of the data using EDA.

In [ ]:
EDA('type')
df.drop(columns=['type'], inplace=True)

In [ ]:
EDA('restriction')

In [ ]:
EDA('reason')

In [ ]:
EDA('zoneAuthority_name')

In [ ]:
EDA('permanent')

In [ ]:
EDA('message')

In [ ]:
df[df['message'] == "Alle vluchten zijn hier verboden op 4 mei / all flights are prohibted on the 4th of may"]

These no-fly zones are related to Remembrance Day. Although they are marked as permanent, they are in fact temporary and can therefore be removed for the purposes of this analysis.

In [ ]:
# delete rows with message "Alle vluchten zijn hier verboden op 4 mei / all flights are prohibted on the 4th of may" 
df = df[df['message'] != "Alle vluchten zijn hier verboden op 4 mei / all flights are prohibted on the 4th of may"]

In [ ]:
EDA('upper_limit')

In [ ]:
EDA('lower_limit')

Drones will operate only within Very Low-Level (VLL) airspace, so areas above 120 meters can be excluded from this analysis.

In [ ]:
len(df)

In [ ]:
# Filter all rows with lower_limit > 120
df = df[df['lower_limit'] <= 120]

In [ ]:
len(df)

In [ ]:
EDA('geometry_type')

In [ ]:
EDA('geometry')

In [ ]:
EDA('circle_radius')    

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='geometry')

In [ ]:
gdf = gdf.set_crs("EPSG:4326")      

In [ ]:
gdf = gdf.to_crs("EPSG:28992") 

In [ ]:
# print geometry types
print(gdf["geometry"].geom_type.value_counts())

Some geometries are in point format but should represent circular zones. We convert these points into circles by creating a multipolygon with a specified radius.

In [ ]:
def convert_points_to_polygons_from_radius(gdf, resolution=128):
    """
    Sets all Point geometries to Polygon, with radius from 'circle_radius'.
    Leaves existing Polygon geometries untouched.
    """
    gdf = gdf.copy()

    if gdf.crs.to_epsg() != 28992:
        print("Warning: CRS is not in meters (EPSG:28992). Please convert first.")

    def convert_geometry(row):
        radius = row.circle_radius
        if row.geometry.geom_type == "Point" and not np.isnan(row.circle_radius):
            return row.geometry.buffer(radius, resolution=resolution)
        else:
            return row.geometry

    gdf["geometry"] = gdf.apply(convert_geometry, axis=1)

    # Update geometry_type field if it exists
    if "geometry_type" in gdf.columns:
        gdf["geometry_type"] = gdf["geometry"].geom_type

    return gdf


In [ ]:
gdf = convert_points_to_polygons_from_radius(gdf)

In [ ]:
gdf = gdf.to_crs("EPSG:4326")

In [ ]:
# Select relevant columns
gdf = gdf[['name', 'message','air_type', 'geometry', 'no_fly']]

gdf.rename(columns={'message': 'description'}, inplace=True)

In [ ]:
# print geometry types
print(gdf["geometry"].geom_type.value_counts())

We also save the dataframe containing all no-fly zones as recorded by the government. This dataset can be used for further research.

In [ ]:
gdf.to_file('../3.no_fly_zones/input/gdf_all_airspace_zones_nl_27feb.geojson', index=False)

In [ ]:
gdf_no_fly_zones = gdf[gdf['no_fly'] == True].copy()
gdf_no_fly_zones['area_type'] = 'no_fly_zone'


In [ ]:
gdf_no_fly_zones.drop(columns=['no_fly'], inplace=True)

In [ ]:
gdf_no_fly_zones.to_file('/Users/cmartens/Documents/thesis_cf_martens/no_fly_zones/input/gdf_no_fly_zones_nl_27feb.geojson', index=False)